In [1]:
from astropy.table import Table
import numpy as np
lens_samples = Table.read('lens_samples.fits')
mask_stack = lens_samples['if_obs_stack'].data
mask_single = lens_samples['if_obs_single'].data

print(np.count_nonzero(mask_stack), np.count_nonzero(mask_single))
#Lens:q_l, re_l, vdisp_l
#Source:q, re_s, mag_g, 

836 728


In [2]:
obs_stack_sample = lens_samples[mask_stack]
# obs_stack_sample.write('csst_obs_stack_samples.fits', overwrite=True)

In [3]:
obs_stack_sample[0]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,bool,bool,bool,int64
0.7551854998370612,121628811912.94075,1.9875,0.09555366191910125,0.510175628241307,4.513799488809944,-0.521156737618329,0.00496968539320269,26.7678,26.8196,26.8888,26.8314,197.12833849063614,0.5375433061960756,0.47409034013176166,0.4384578425252495,22.522684695449517,20.608330225394962,19.775954598753057,19.2866136257531,46.301281851665344,32.09652462512723,23.398015788489875,16.10893871790978,57.95248174133211,g,3.3082721934494015,False,True,True,10


In [4]:
np.random.seed(1)
hist_size = 10000

stacked_samples = lens_samples[mask_stack]
indices = np.random.choice(len(stacked_samples), size=hist_size, replace=False)
stacked_samples_ = stacked_samples[indices]

vdisp_l = stacked_samples_['vdisp_l']
q_l = stacked_samples_['q_l']
re_l = stacked_samples_['re_l']

q_s = stacked_samples_['q_s0']
re_s = stacked_samples_['re_s0']
mag_g_s = stacked_samples_['mag_g_s0']

from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=50, 
    zl_max=2.5, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
lens_pop.draw_lens_samples(
    nsamples=hist_size,
    nsrcs_per_sample=1,
    src_over_density=1,
)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [14]:
from scipy.ndimage import gaussian_filter1d
def smooth_hist(data, bins=30, smooth=True, sigma=2, log_bin=False, density=False):
    if log_bin:
        bins = np.logspace(np.log10(np.min(data)), np.log10(np.max(data)),  bins)
        
    counts, bins = np.histogram(data, bins=bins, density=density)
    
    if smooth:
        counts_smoothed = gaussian_filter1d(counts, sigma=sigma)
    else:
        counts_smoothed = counts
        
    #for beautiful step plot
    counts_smoothed = np.insert(counts_smoothed, 0, 0)
    bins = np.insert(bins, 0, bins[0]-1e-3)
    counts_smoothed = np.append(counts_smoothed, 0)
    bins = np.append(bins, bins[-1]+1e-3)
    return counts_smoothed, bins

In [ ]:
from matplotlib import pyplot as plt
from astropy.visualization import hist
import scienceplots
plt.style.use(['science','no-latex', 'nature'])
# Set the font family and size for all text
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
# Set the background color and grid style
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.linewidth'] = 1.5
plt.rcParams['grid.color'] = 'black'
plt.rcParams['grid.alpha'] = 0.5

columnwidth = 3.33
aspect_ratio = 0.8
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(3*columnwidth+2*0.5, 2*columnwidth*aspect_ratio+1))

this_ax = axes[0,0]
data = lens_pop.dfl_pop.q_arr
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent ETG", ls='--')
data = q_l
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Forground deflector")
this_ax.set_xlabel(r'$q_\mathrm{l}$')
this_ax.set_ylabel(r'$\rm PDF$')
_, ymax = this_ax.get_ylim()
this_ax.set_ylim(0,ymax)
this_ax.set_xlim(0.2,1.0)
this_ax.set_yticklabels([])
this_ax.legend(loc='upper left')


this_ax = axes[0,1]
data = lens_pop.dfl_pop.Re_arr
counts_smoothed, bins = smooth_hist(data, density=True, log_bin=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent ETG", ls='--')
data = re_l
counts_smoothed, bins = smooth_hist(data, density=True, log_bin=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Forground deflector")
this_ax.set_xlabel(r'$R_\mathrm{e}^\mathrm{l} \; \left(\mathrm{arcsec}\right) $')
_, ymax = this_ax.get_ylim()
this_ax.set_xscale('log')
this_ax.set_ylim(0,ymax)
this_ax.set_yticklabels([])
this_ax.legend()


this_ax = axes[0,2]
data = lens_pop.dfl_pop.vdisp_arr
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent ETG", ls='--')
data = vdisp_l
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Forground deflector")
this_ax.set_xlabel(r'$\sigma_v \; \left(\mathrm{km/s}\right)$')
_, ymax = this_ax.get_ylim()
this_ax.set_ylim(0,ymax)
this_ax.set_yticklabels([])
this_ax.legend()


this_ax = axes[1,0]
data = lens_pop.src_pop.q_arr.reshape(-1)
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent source", ls='--')
data = q_s
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Background source")
this_ax.set_xlabel(r'$q_\mathrm{s}$')
this_ax.set_ylabel(r'$\rm PDF$')
_, ymax = this_ax.get_ylim()
this_ax.set_ylim(0,ymax)
this_ax.set_yticklabels([])
this_ax.set_xlim(0.2,1.0)
this_ax.legend(loc='lower right')


this_ax = axes[1,1]
data = lens_pop.src_pop.Re_arr.reshape(-1)
counts_smoothed, bins = smooth_hist(data, density=True, log_bin=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent source", ls='--')
data = re_s
counts_smoothed, bins = smooth_hist(data, density=True, log_bin=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Background source")
this_ax.set_xlabel(r'$R_\mathrm{e}^\mathrm{s} \; \left(\mathrm{arcsec}\right)$')
_, ymax = this_ax.get_ylim()
this_ax.set_xscale('log')
this_ax.set_ylim(0,ymax)
this_ax.set_yticklabels([])
this_ax.legend()

this_ax = axes[1,2]
data = lens_pop.src_pop.app_mag_arr['g'].reshape(-1)
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Parent source", ls='--')
data = mag_g_s
counts_smoothed, bins = smooth_hist(data, density=True)
this_ax.step(bins[:-1], counts_smoothed,  where='post', color='black', label="Background source")
this_ax.set_xlabel(r'$\mathrm{mag}_\mathrm{g}^\mathrm{src}$')
_, ymax = this_ax.get_ylim()
this_ax.set_ylim(0,ymax)
this_ax.set_yticklabels([])
this_ax.legend(loc='upper left')

plt.tight_layout()
fig.savefig('figures/selection_effect.pdf', bbox_inches='tight')
plt.close(fig)
# plt.show()

In [7]:
lens_pop.dfl_pop.vdisp_arr

array([221.19507154,  94.92198932, 202.10574299, ..., 122.71728187,
       130.06109923, 163.0703084 ])